In [39]:
import fitz #pymupdf
from collections import defaultdict
import json, os
from PIL import Image, ImageDraw

pdf_path = r"C:\Users\dkhun\UC Davis\AISC Github repository\BeginnerProjectFallQuarter2025\data\labeled_pdfs\Manuh_Africa_in_Contemporary_Perspective_A_Textboo..._----_(SECTION_1_AFRICA GEOGRAPHY,_POPULATION_&_LANGUAGE).pdf"
input_file = fitz.open(pdf_path)
page = input_file[0]
words = page.get_text("words")
# words is list of tuples: (x0, y0, x1, y1, "word", block_no, line_no, word_no)

#use defaultdict instead of {} for optimization because it allows for better appending
lines = defaultdict(list)
for x0, y0, x1, y1, word, block_no, line_no, word_no in words:
    lines[(block_no, line_no)].append((x0, y0, x1, y1, word, word_no)) #extra parenthesis used to indicate tuple

line_boxes = []     #Will contain tuples of (x0, y0, x1, y1, 'text within line')
for (block_no, line_no), word_list in lines.items():
    word_list_sorted = sorted(word_list, key = lambda w:w[0])   #Sorts each tuple w based on w[0], which is the left pixel x0
    left = min(w[0] for w in word_list_sorted)
    top = min(w[1] for w in word_list_sorted)
    right = max(w[2] for w in word_list_sorted)
    bottom = max(w[3] for w in word_list_sorted)
    text_in_line = " ".join(w[4] for w in word_list_sorted)

    line_boxes.append((left, top, right, bottom, text_in_line))

# for x0, y0, x1, y1, text in line_boxes:
#     box = fitz.Rect(x0, y0, x1, y1)   #Creates a rectangle for each line
#     page.add_rect_annot(box)

# Render page to image
dpi = 300
zoom = dpi / 72 #scaling factor because pymupdf uses dpi = 72

mat = fitz.Matrix(zoom, zoom)
pix = page.get_pixmap(matrix=mat)
mode = "RGB" if pix.n < 4 else "RGBA"
img = Image.frombytes(mode, (pix.width, pix.height), pix.samples)

# Draw rectangles for each line
draw = ImageDraw.Draw(img)
for x0, y0, x1, y1, text in line_boxes:
    draw.rectangle([x0*zoom, y0*zoom, x1*zoom, y1*zoom], outline="red", width=2)

# Show the image with bounding boxes
img.show()


input_file.close()
